# Dataset : Brain Tumor Classification MRI

In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tqdm import tqdm
import os
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
classes={'glioma_tumor': 0, 'meningioma_tumor': 1, 'no_tumor':2, 'pituitary_tumor':3}
targetsTrain=[]
#targetsTest=[]
dataTrain=[]
#dataTest=[]
import os
for dirname, _, filenames in os.walk('../input/brain-tumor-classification-mri'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        if 'Training' in dirname:
            img = cv2.imread(os.path.join(dirname, filename))
            img = cv2.resize(img,(120, 120))
                             
            dataTrain.append(np.array(img))
            
            for key, value in classes.items(): 
                if key in dirname:
                    targetsTrain.append(classes[key]) 
        else:
            img = cv2.imread(os.path.join(dirname, filename))
            img = cv2.resize(img,(120, 120))
            dataTrain.append(np.array(img))
            for key, value in classes.items(): 
                if key in dirname:
                    targetsTrain.append(classes[key]) 

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [17]:
dataTrain = np.array(dataTrain)
targetsTrain = np.array(targetsTrain)

In [18]:
dataTrain, targetsTrain = shuffle(dataTrain, targetsTrain, random_state=101)

# Split dataset

In [21]:
from sklearn.model_selection import train_test_split
dataTrain, dataTest, targetsTrain, targetsTest = train_test_split(dataTrain, targetsTrain, test_size=0.2,random_state=101)

# Creating model and adding layers

In [24]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(120, 120, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(4))

In [25]:
y_train = tf.keras.utils.to_categorical(targetsTrain)

# Compiling the model

In [28]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Fitting the model

In [29]:
history = model.fit(dataTrain, targetsTrain, epochs=20, batch_size=32
                    #validation_data=()
                   )

# Evaluating the model

In [30]:
score = model.evaluate(dataTest, targetsTest, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [32]:
#enregistrer le modele 
from keras.models import load_model
model.save('MYmodel.h5')  # creates a HDF5 file 'MYmodel.h5'